In [1]:
import datasets

In [2]:
import transformers

In [3]:
from bg2vec.arguments import *
from bg2vec.data_util import *
from bg2vec.model import *
from bg2vec.training import *

/home/mboyanov/miniconda3/lib/python3.11/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [4]:
import os 

In [5]:
model_args, data_args, training_args, custom_args = parser.parse_json_file("model_configurations/bggpt-7b.json")

config_kwargs = {
    "cache_dir": model_args.cache_dir,
    "revision": model_args.model_revision,
    "token": model_args.token,
    "trust_remote_code": model_args.trust_remote_code,
}
if training_args.gradient_checkpointing:
    training_args.gradient_checkpointing_kwargs = {"use_reentrant": False}
# Set seed before initializing model.

transformers.set_seed(training_args.seed)

## Loading the model

In [ ]:
from transformers import (
    AutoConfig)

config = AutoConfig.from_pretrained(
    model_args.model_name_or_path, **config_kwargs
)
model_class = get_model_class(config)
model_class

In [ ]:
torch_dtype = (
    model_args.torch_dtype
    if model_args.torch_dtype in ["auto", None]
    else getattr(torch, model_args.torch_dtype)
)
model = model_class.from_pretrained(
    model_args.model_name_or_path,
    from_tf=bool(".ckpt" in model_args.model_name_or_path),
    config=config,
    cache_dir=model_args.cache_dir,
    revision=model_args.model_revision,
    token=model_args.token,
    trust_remote_code=model_args.trust_remote_code,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=model_args.low_cpu_mem_usage,
    attn_implementation=model_args.attn_implementation,
)

In [ ]:
model

In [ ]:
model.model.layers[0].self_attn.is_causal

In [ ]:
model = initialize_peft(
    model,
    lora_r=custom_args.lora_r,
    lora_alpha=2 * custom_args.lora_r,
    lora_dropout=custom_args.lora_dropout,
)

In [ ]:
model.model.base_model.model.layers[0]

## Setting up data collation

In [ ]:
from transformers import AutoTokenizer
tokenizer_kwargs = {
    "cache_dir": model_args.cache_dir,
    "use_fast": model_args.use_fast_tokenizer,
    "revision": model_args.model_revision,
    "token": model_args.token,
    "trust_remote_code": model_args.trust_remote_code,
}
tokenizer = AutoTokenizer.from_pretrained(
        model_args.model_name_or_path, **tokenizer_kwargs
)
tokenizer

In [ ]:
if tokenizer.mask_token is None:
    if custom_args.mask_token_type == "blank":
        tokenizer.mask_token = "_"
    elif custom_args.mask_token_type == "eos":
        tokenizer.mask_token = tokenizer.eos_token
    elif custom_args.mask_token_type == "mask":
        tokenizer.add_tokens(["<mask>"])
        tokenizer.mask_token = "<mask>"
    else:
        raise ValueError(
            f"mask_token_type {custom_args.mask_token_type} is not supported."
        )

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
data_collator = DataCollatorForLanguageModelingWithFullMasking(
    tokenizer=tokenizer,
    mlm_probability=data_args.mlm_probability
)

In [ ]:
import torch

In [ ]:
data_collator.tokenizer.vocab['_']

In [ ]:
data_collator( (torch.randint(0, 10, (1, 10)), ))

## Loading up the dataset

In [ ]:
tokenized_datasets = datasets.load_from_disk("grouped_512")

In [ ]:
train_dataset = tokenized_datasets["train"]
if data_args.max_train_samples is not None:
    max_train_samples = min(len(train_dataset), data_args.max_train_samples)
    train_dataset = train_dataset.select(range(max_train_samples))

In [ ]:
eval_dataset = tokenized_datasets["validation"]
if data_args.max_eval_samples is not None:
    max_eval_samples = min(len(eval_dataset), data_args.max_eval_samples)
    eval_dataset = eval_dataset.select(range(max_eval_samples))

## Setting up the trainer

In [6]:
from transformers import is_torch_tpu_available
import evaluate
from bg2vec.training import MNTPTrainer

In [7]:
from bg2vec.metrics import *

evaluator = MetricEvaluator(model_args.cache_dir)

In [ ]:
trainer = MNTPTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset if training_args.do_train else None,
    eval_dataset=eval_dataset if training_args.do_eval else None,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=evaluator if training_args.do_eval and not is_torch_tpu_available()
                              else None,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics
    if training_args.do_eval and not is_torch_tpu_available()
    else None,

)

trainer.add_callback(StopTrainingCallback(custom_args.stop_after_n_steps))
trainer.callback_handler.remove_callback(transformers.integrations.integration_utils.WandbCallback)

## Training

In [ ]:
train_result = trainer.train()    

## Model export

In [ ]:
model.save_pretrained("bggpt-mntp-pretrained-2")

In [ ]:
trainer.save_model()  # Saves the tokenizer too for easy upload
metrics = train_result.metrics

max_train_samples = (
    data_args.max_train_samples
    if data_args.max_train_samples is not None
    else len(train_dataset)
)
metrics["train_samples"] = min(max_train_samples, len(train_dataset))

trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()